In [1]:
        

import numpy as np # numpy

from sklearn import datasets # load dataset
from sklearn.model_selection import train_test_split # split dataset
from sklearn.preprocessing import StandardScaler # standard scaler
from sklearn.svm import SVC # import model
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score # check accuracy
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn import preprocessing

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_graph(list_of_x, list_of_y, labelX, labelY):
    plt.plot(list_of_x, list_of_y, color="red", marker="o",  linestyle="")
    ax = plt.gca()
    ax.tick_params(axis='y', colors='red')
    ax.tick_params(axis='x', colors='red')
    
    ax.set_xlabel(labelX)
    ax.xaxis.label.set_color('red')

    ax.set_ylabel(labelY)
    ax.yaxis.label.set_color('red')
    
    plt.xticks(np.arange(min(list_of_x) * (-1), max(list_of_x), 1))
    plt.yticks(np.arange(0, 1, 0.1))
    plt.show()
    

def replicate_classes(X, y):

    X_class_minoritary = X[y==1] #pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
    y_class_minoritary = y[y==1]
    
    conX = np.concatenate((X, X_class_minoritary), axis=0)
    
    conY = np.concatenate((y, y_class_minoritary), axis=0)
    
    return conX, conY

In [2]:
def get_sigmaE(vectorX, vectorW):
    """
        Multiplies the input (vectorX) by the weights (vectorW), resulting in a diagonal matrix. 
        It discards any imaginary part vectorX and vectorW might have.
        Equivalent of Equation #17 in the Article.
    """
    n = len(vectorX)
    sigmaE = np.zeros((n,n))
    for i in range(n):
        sigmaE[i,i] = np.real(vectorX[i])*np.real(vectorW[i])

    return sigmaE

def get_sigmaQ(n):
    """
        Sums sigmaX, sigmaY and sigmaZ to get sigmaQ.
        - sigmaX comes from Equation #7 = [0, 1   1, 0]
        - sigmaY comes from Equation #8 = [0, -i  i, 0]
        - sigmaZ comes from Equation #9 = [1, 0   0, -1]
        Equivalent of Equation #16 in the Article.
    """
    sigmaQ = np.zeros((n,n))
    sigmaX = np.array([[0,1], [1,0]])
    sigmaY = np.array([[0,-1j], [1j,0]])
    sigmaZ = np.array([[1,0], [0,-1]])
    sigmaQ = sigmaX + sigmaY + sigmaZ

    return sigmaQ

def get_sigmaQ_param(n, param):
    """
        Sums sigmaX, sigmaY and sigmaZ to get sigmaQ.
        - sigmaX comes from Equation #7 = [0, 1   1, 0]
        - sigmaY comes from Equation #8 = [0, -i  i, 0]
        - sigmaZ comes from Equation #9 = [1, 0   0, -1]
        Equivalent of Equation #16 in the Article.
    """
    sigmaQ = np.zeros((n,n))
    sigmaX = np.array([[0,1], [1,0]])
    sigmaY = np.array([[0,-1j], [1j,0]])
    sigmaZ = np.array([[1,0], [0,-1]])
    sigmaH = np.array([[1,1],[1,-1]])*(1/np.sqrt(2))
    sigmaQ = param[0]*sigmaX + param[1]*sigmaY + param[2]*sigmaZ + param[3]*sigmaH

    return sigmaQ


def get_U_operator(sigmaQ, sigmaE):
    """
        Makes the exponential matrix of tensor product between sigmaQ and sigmaE and multiplies it by j. 
        Equivalent of Equation #15 in the Article.
    """
    sigmaQ[np.isnan(sigmaQ)] = 0
    sigmaE[np.isnan(sigmaE)] = 0
    return np.matrix(expMatrix(1j*np.kron(sigmaQ, sigmaE)))

def get_p(psi):
    """
        Creates a matrix out of psi and multiply it against its inverse, 
        resulting in a column vector in the form [[alfa]. [beta]].
        Does the operation |psi><psi| from Equation #18 or #19 in the Article.
    """
    psi = np.matrix(psi)
    return psi * psi.getH()



def create_and_execute_classifier_ORIGINAL(vectorX, vectorW, normalizeX=False, normalizeW=False, paramsClassifier=[1,1,1,0]):
    """
        Applies the ICQ classifier using only the math behind the Quantum Classifier 
        described in Interactive Quantum Classifier Inspired by Quantum Open System Theory
        article. 
        After doing so, it gets the result of Equation #20 and returns Z as the predicted class and
        the probability of being the class 1.
        Works only for binary classifications, therefore, if the probability of class 0 is needed, it can
        be 1 - probability of being class 1.
    """
    params = paramsClassifier

    if normalizeX:
        vectorX = normalize(vectorX)
    if normalizeW:
        vectorW = normalize(vectorW)

    # Eq #16
    #sigmaQ = get_sigmaQ(2)
    sigmaQ = get_sigmaQ_param(2, params)

    # Eq #17
    sigmaE = get_sigmaE(vectorX, vectorW)

    # Eq #15
    U_operator = get_U_operator(sigmaQ, sigmaE)

    # Eq #18 applied on a Quantum state equivalent of Hadamard(|0>) = 1/sqrt(2) * (|0> + |1>) 
    p_cog = get_p([[1/np.sqrt(2)],[1/np.sqrt(2)]])

    # As we must have 1 row per attribute of the input, we need env to be as big as one instance of our input
    N = len(vectorX)

    # Eq #19 applied on a Quantum state equivalent of Hadamard(|000000...>) = 1/sqrt(N) * (|000000...> + ... + |11111111....>) 
    p_env = get_p([[1/np.sqrt(N)] for i in range(N)])

    # First part of Equation #20 in the Article
    quantum_operation = np.array(U_operator * (np.kron(p_cog, p_env)) * U_operator.getH())

    # Second part of Equation #20 in the Article
    p_cog_new = np.trace(quantum_operation.reshape([2,N,2,N]), axis1=1, axis2=3)

    # As the result is a diagonal matrix, the probability of being class 0 will be on position 0,0
    p_cog_new_00_2 = p_cog_new[0,0]

    # ... and the probability of being class 1 will be on position 1,1
    p_cog_new_11_2 = p_cog_new[1,1]
    if (p_cog_new_00_2 >= p_cog_new_11_2):
        z = 0
    else:
        z = 1
    return z, p_cog_new_11_2, U_operator

def create_and_execute_classifier_NEW(vectorX, vectorW, normalizeX=False, normalizeW=False):
    """
        Applies the ICQ classifier using only the math behind the Quantum Classifier 
        described in Interactive Quantum Classifier Inspired by Quantum Open System Theory
        article. 
        After doing so, it gets the result of Equation #20 and returns Z as the predicted class and
        the probability of being the class 1.
        Works only for binary classifications, therefore, if the probability of class 0 is needed, it can
        be 1 - probability of being class 1.
    """

    global params
    if normalizeX:
        vectorX = normalize(vectorX)
    if normalizeW:
        vectorW = normalize(vectorW)  
    # Eq #16
    #sigmaQ = get_sigmaQ(2)
    #alfa, beta, delta, gama = params # [Symbol(chr(ord('X') + i)) for i in range(1,5)]
    #sigmaQ = np.matrix([
    #     [exp(1j*(alfa-beta/2-delta/2))*cos(gama/2), exp(1j*(alfa-beta/2+delta/2))*sin(gama/2)],
    #    [exp(1j*(alfa+beta/2-delta/2))*sin(gama/2), exp(1j*(alfa+beta/2+delta/2))*cos(gama/2)]
    #    ])
    #theta, phi, delta = params # [Symbol(chr(ord('X') + i)) for i in range(1,5)]
    #sigmaQ = np.matrix([
    #     [cos(theta/2), -exp(1j*delta)*sin(theta/2)],
    #     [exp(1j*phi)*sin(theta/2), exp(1j*(phi+delta))*cos(theta/2)]
    #    ])
    sigmaQ = get_sigmaQ_param(2, params)

    #alfa, beta, gama, delta = params
    #sigmaE = np.matrix([
    #     [exp(1j*(alfa-beta/2-delta/2))*cos(gama/2), exp(1j*(alfa-beta/2+delta/2))*sin(gama/2)],
    #     [exp(1j*(alfa+beta/2-delta/2))*sin(gama/2), exp(1j*(alfa+beta/2+delta/2))*cos(gama/2)]
    #    ])

    # Eq #17
    #sigmaE = get_sigmaE(vectorX, vectorW)

    N = len(vectorX)
    # Eq #15
    sigmaE = np.zeros((N,N))
    for i in range(N):
        sigmaE[i,i] = vectorW[i]
        
    U_operator = get_U_operator(sigmaQ, sigmaE)

    # Eq #18 applied on a Quantum state equivalent of Hadamard(|0>) = 1/sqrt(2) * (|0> + |1>) 
    p_cog = get_p([[1/np.sqrt(2)],[1/np.sqrt(2)]])

    # As we must have 1 row per attribute of the input, we need env to be as big as one instance of our input
    

    # Eq #19 applied on a Quantum state equivalent of Hadamard(|000000...>) = 1/sqrt(N) * (|000000...> + ... + |11111111....>) 
    vectorX = normalize(vectorX)
    p_env = get_p([[vectorX_i] for vectorX_i in vectorX])

    # First part of Equation #20 in the Article
    quantum_operation = np.array(U_operator * (np.kron(p_cog, p_env)) * U_operator.getH())

    # Second part of Equation #20 in the Article
    p_cog_new = np.trace(quantum_operation.reshape([2,N,2,N]), axis1=1, axis2=3)

    # As the result is a diagonal matrix, the probability of being class 0 will be on position 0,0
    p_cog_new_00_2 = p_cog_new[0,0]

    # ... and the probability of being class 1 will be on position 1,1
    p_cog_new_11_2 = p_cog_new[1,1]
    if (p_cog_new_00_2 >= p_cog_new_11_2):
        z = 0
    else:
        z = 1
    return z, p_cog_new_11_2, U_operator

def update_weights(weights, y, z, x, p, n):
  """
    Updates the weights. Equation #34 in the Article.

    y is the expected classification [0, 1];
    z is the actual classification [0, 1];
    x is the attribute vector;
    p is the probability of the class 1 (0, 1), powered to 2 (p²);
    n is the learning rate.
  """
  # Eq 33
  loss_derivative_on_weight = (1-p)*x

  # Eq 34
  weights = weights-n*(z-y)*loss_derivative_on_weight
  weights[np.isnan(weights)] = 0
  return weights


def get_prob_1(hist):
    count0 = 0
    count1 = 0
    if '0' in hist:
        count0 = hist['0']
    if '1' in hist:
        count1 = hist['1']
    return count1/(count1+count0)

def normalize(x):
    v_norm = x / (np.linalg.norm(x) + 1e-16)
    return v_norm

def update_weights(weights, y, z, x, p, n):
  """
    Updates the weights. Equation #34 in the Article.
    
    y is the expected classification [0, 1];
    z is the actual classification [0, 1];
    x is the attribute vector;
    p is the probability of the class 1 (0, 1), powered to 2 (p²);
    n is the learning rate.
  """
  # Eq 33
  loss_derivative_on_weight = (1-p)*x

  # Eq 34
  weights = weights-n*(z-y)*loss_derivative_on_weight
  weights[np.isnan(weights)] = 0
  return weights

In [3]:
import numpy as np
from scipy.linalg import expm as expMatrix

import numpy as np
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from sklearn.metrics import f1_score


class TemplateClassifier(ClassifierMixin, BaseEstimator):
    """ An example classifier which implements a 1-NN algorithm.
    For more information regarding how to build your own classifier, read more
    in the :ref:`User Guide <user_guide>`.
    Parameters
    ----------
    demo_param : str, default='demo'
        A parameter used for demonstation of how to pass and store paramters.
    Attributes
    ----------
    X_ : ndarray, shape (n_samples, n_features)
        The input passed during :meth:`fit`.
    y_ : ndarray, shape (n_samples,)
        The labels passed during :meth:`fit`.
    classes_ : ndarray, shape (n_classes,)
        The classes seen at :meth:`fit`.
    """
    def __init__(self, classifierFunction, rate_succ=0.2, paramsClassifier=None, MAX_ITER=1000, demo_param='demo'):
        self.demo_param = demo_param
        self.rate_succ = rate_succ
        self.classifierFunction = classifierFunction
        self.MAX_ITER = MAX_ITER
        self.paramsClassifier = paramsClassifier

    def fit(self, X, y):
        """A reference implementation of a fitting function for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,)
            The target values. An array of int.
        Returns
        -------
        self : object
            Returns self.
        """
        np.random.seed(1)
        
        X,y = replicate_classes(X, y)
            
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        
        low  = -1
        high = 1
        dimensions = len(X[0])

        rate=0
        
        weight = np.random.uniform(low=low, high=high, size=(dimensions,))
        
        ITERATION = 0
        
        bestWeight = []
        bestRate = 0.0
        self.ratesDuringTraining = []
        
        
        while rate < self.rate_succ and ITERATION < self.MAX_ITER:
            rate=0
            #training step
            for x_train, y_train in zip(X, y):
                z, p_cog, qo = self.classifierFunction(x_train, weight, self.paramsClassifier)
                weight = update_weights(weight, y_train, z, x_train, p_cog, n=0.01)
                
            for x_train, y_train in zip(X, y):
                z, p_cog, qo = self.classifierFunction(x_train, weight, self.paramsClassifier)            
                if z == y_train:
                    rate +=1
            
            rate = rate/len(X)
            self.ratesDuringTraining.append(rate)
            ITERATION += 1
            if (rate >= bestRate):
                bestWeight = weight
                bestRate = rate
            if ITERATION%1000 == 0:
                weight = np.random.uniform(low=low, high=high, size=(dimensions,))
            
        self.rate = bestRate
        self.weight_ = bestWeight
        self.X_ = X
        self.y_ = y
        print("best weight", bestWeight)
        print("best rate", bestRate)
        plot_graph(range(ITERATION), self.ratesDuringTraining , "iter", "rate")
        # Return the classifier
        return self

    def predict(self, X):
        """ A reference implementation of a prediction for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            The label for each sample is the label of the closest sample
            seen during fit.
        """
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_', 'weight_'])

        # Input validation
        X = check_array(X)
        
        outputs = []
        for x in X:                   
            z, p_cog, qo = self.classifierFunction(x, self.weight_, self.paramsClassifier)
            outputs.append(z)
                               
        return outputs

    def predict_proba(self, X):
        outputs = []
        for x in X:                   
            z, p_cog, qo = self.classifierFunction(x, self.weight_, self.paramsClassifier)
            outputs.append([1-p_cog.real, p_cog.real])
                               
        return np.array(outputs)

def executeIris(randomState = 1, classifierFunction, paramsClassifier, MAX_ITER ):
    # load dataset
    iris = datasets.load_iris()
    X = iris.data[:, [0,1,2, 3]]
    y = iris.target

    # split training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = randomState, stratify = y)


    # apply MinMaxScaler for iris data set, [0, 1] for the range
    from sklearn.preprocessing import MinMaxScaler
    normalized_X_train = preprocessing.normalize(X_train)
    normalized_X_test  = preprocessing.normalize(X_test)
    
    clf = OneVsRestClassifier(
        TemplateClassifier(
            classifierFunction=classifierFunction, 
            paramsClassifier = paramsClassifier,
            MAX_ITER=MAX_ITER,
            rate_succ=1.0)).fit(normalized_X_train, y_train)

    score = clf.score(normalized_X_test, y_test)
    f1score = f1_score(clf.predict(normalized_X_test), y_test, average='macro')

    return score, f1score

In [19]:
# load dataset
iris = datasets.load_iris()
X = iris.data[:, [0,1,2, 3]]
y = iris.target
print('Class label :', np.unique(y))


# split training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1, stratify = y)

# check unique value's count
print('y label count :', np.bincount(y))
print('y_train label count :', np.bincount(y_train))
print('y_test label count :', np.bincount(y_test))


# standardize
#sc = StandardScaler()
#sc.fit(X_train) # calculate mu and sigma
#X_train_std = sc.transform(X_train) # standardize
#X_test_std = sc.transform(X_test)

# apply MinMaxScaler for iris data set, [0, 1] for the range
from sklearn.preprocessing import MinMaxScaler
normalized_X_train = preprocessing.normalize(X_train)
normalized_X_test  = preprocessing.normalize(X_test)

Class label : [0 1 2]
y label count : [50 50 50]
y_train label count : [45 45 45]
y_test label count : [5 5 5]


In [21]:
normalized_X_train

array([[0.74088576, 0.33173989, 0.55289982, 0.18798594],
       [0.67767924, 0.32715549, 0.59589036, 0.28041899],
       [0.71562645, 0.3523084 , 0.56149152, 0.22019275],
       [0.72415258, 0.32534391, 0.56672811, 0.22039426],
       [0.80779568, 0.53853046, 0.23758697, 0.03167826],
       [0.77577075, 0.60712493, 0.16864581, 0.03372916],
       [0.71576546, 0.30196356, 0.59274328, 0.21249287],
       [0.76521855, 0.33391355, 0.52869645, 0.15304371],
       [0.73544284, 0.35458851, 0.55158213, 0.1707278 ],
       [0.73154399, 0.28501714, 0.57953485, 0.21851314],
       [0.74143307, 0.29421947, 0.57667016, 0.17653168],
       [0.69052512, 0.32145135, 0.60718588, 0.22620651],
       [0.82512295, 0.52807869, 0.19802951, 0.03300492],
       [0.76467269, 0.31486523, 0.53976896, 0.15743261],
       [0.78591858, 0.57017622, 0.23115252, 0.06164067],
       [0.69589887, 0.34794944, 0.57629125, 0.25008866],
       [0.80533308, 0.54831188, 0.2227517 , 0.03426949],
       [0.79594782, 0.55370283,

In [11]:
%%time
#clf = OneVsOneClassifier(TemplateClassifier(classifierFunction=create_and_execute_classifier, rate_succ=0.4)).fit(X_train, y_train)


clf = OneVsRestClassifier(
    TemplateClassifier(
        classifierFunction=create_and_execute_classifier_ORIGINAL, 
        paramsClassifier = [1,1,1,0],
        MAX_ITER=3000,
        rate_succ=1.0)).fit(normalized_X_train, y_train)

#clf.predict(normalized_X_test[:10])
#y_test[:10]
print("SCORE", clf.score(normalized_X_test, y_test))
print("F-1 Score Macro", f1_score(clf.predict(normalized_X_test), y_test, average='macro'))

KeyboardInterrupt: 

In [12]:
%%time
#clf = OneVsOneClassifier(TemplateClassifier(classifierFunction=create_and_execute_classifier, rate_succ=0.4)).fit(X_train, y_train)


clf = OneVsRestClassifier(
    TemplateClassifier(
        classifierFunction=create_and_execute_classifier_ORIGINAL, 
        paramsClassifier = [0.2,0.9,0.3,0.5],
        MAX_ITER=3000,
        rate_succ=1.00)).fit(normalized_X_train, y_train)

#clf.predict(normalized_X_test[:10])
#y_test[:10]
print("SCORE", clf.score(normalized_X_test, y_test))
print("F-1 Score Macro", f1_score(clf.predict(normalized_X_test), y_test, average='macro'))

KeyboardInterrupt: 

In [13]:
%%time
#clf = OneVsOneClassifier(TemplateClassifier(classifierFunction=create_and_execute_classifier, rate_succ=0.4)).fit(X_train, y_train)


clf = OneVsRestClassifier(
    TemplateClassifier(
        classifierFunction=create_and_execute_classifier_NEW, 
        paramsClassifier = [1,1,1,0],
        MAX_ITER=3000,
        rate_succ=1.0)).fit(normalized_X_train, y_train)

#clf.predict(normalized_X_test[:10])
#y_test[:10]
print("SCORE", clf.score(normalized_X_test, y_test))
print("F-1 Score Macro", f1_score(clf.predict(normalized_X_test), y_test, average='macro'))

NameError: name 'params' is not defined

In [14]:
%%time
#clf = OneVsOneClassifier(TemplateClassifier(classifierFunction=create_and_execute_classifier, rate_succ=0.4)).fit(X_train, y_train)


clf = OneVsRestClassifier(
    TemplateClassifier(
        classifierFunction=create_and_execute_classifier_NEW, 
        paramsClassifier = [0.2,0.9,0.3,0.5],
        MAX_ITER=3000,
        rate_succ=1.00)).fit(normalized_X_train, y_train)

#clf.predict(normalized_X_test[:10])
#y_test[:10]
print("SCORE", clf.score(normalized_X_test, y_test))
print("F-1 Score Macro", f1_score(clf.predict(normalized_X_test), y_test, average='macro'))

NameError: name 'params' is not defined